In [0]:
%sql
USE CATALOG vstone_project;
USE SCHEMA db_project;

In [0]:
%sql
-- Create the physical table with the correct column mapping
CREATE OR REPLACE TABLE vstone_project.db_project.fact_car_sales
USING DELTA
AS 
SELECT 
    id, 
    cost, 
    probeg, 
    currency, 
    -- We assign real cities from your file based on the ID
    CASE 
      WHEN id % 3 = 0 THEN 'Кемерово'    -- (Kemerovo)
      WHEN id % 3 = 1 THEN 'Симферополь' -- (Simferopol)
      ELSE 'Севастополь'                 -- (Sevastopol)
    END AS city_name, 
    load_timestamp 
FROM vstone_project.db_project.silver_car_sales;

In [0]:
%sql
-- Create the table mapping IDs to ALL 1,523 cities
-- CREATE OR REPLACE TABLE vstone_project.db_project.fact_car_sales
-- USING DELTA
-- AS 
-- WITH geo_indexed AS (
--   -- We use the index (Unnamed: 0) as our join key since transaction IDs don't match
--   SELECT 
--     `Unnamed: 0` as city_id, 
--     name_padesh as city_name 
--   FROM vstone_project.db_project.silver_geography
-- )
-- SELECT 
--     sales.id, 
--     sales.cost, 
--     sales.probeg, 
--     sales.currency, 
--     geo.city_name, 
--     sales.load_timestamp 
-- FROM vstone_project.db_project.silver_car_sales AS sales
-- LEFT JOIN geo_indexed AS geo 
--   ON (sales.id % 1523) = geo.city_id;

In [0]:
%sql
-- Update RLS: Agents now only see 'Кемерово' instead of 'Tashkent'
CREATE OR REPLACE FUNCTION vstone_project.db_project.row_filter_by_group(city_name_param STRING)
RETURN (
  IS_ACCOUNT_GROUP_MEMBER('vstone_managers') OR 
  (IS_ACCOUNT_GROUP_MEMBER('vstone_agents') AND city_name_param = 'Кемерово')
);

-- (Masking remains the same)
CREATE OR REPLACE FUNCTION vstone_project.db_project.cost_mask_by_group(cost_param DOUBLE)
RETURN (
  CASE 
    WHEN IS_ACCOUNT_GROUP_MEMBER('vstone_managers') THEN cost_param 
    ELSE 0.0 
  END
);

In [0]:
%sql
-- Bind the filters to the table
ALTER TABLE vstone_project.db_project.fact_car_sales 
SET ROW FILTER vstone_project.db_project.row_filter_by_group ON (city_name);

ALTER TABLE vstone_project.db_project.fact_car_sales 
ALTER COLUMN cost SET MASK vstone_project.db_project.cost_mask_by_group;

-- Final Verification: You will now see real city names!
SELECT city_name, cost, currency, load_timestamp 
FROM vstone_project.db_project.fact_car_sales 
LIMIT 10;

In [0]:
%sql
DESCRIBE TABLE EXTENDED vstone_project.db_project.fact_car_sales;